# Tutorial 3: Sparse Spyker
We saw how to use spyker in the previous tutorials. Here, we show how to change previous tutorials to use the sparse interface of the library. As before, we import the needed tools first.

In [ ]:
import spyker, torch
from spyker import sparse
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from torch.utils.data import TensorDataset, DataLoader

The dataset is loaded like before.

In [ ]:
batch, root = 64, './MNIST'
trainx, trainy, testx, testy = spyker.read_mnist(
    root+'/train-images-idx3-ubyte', root+'/train-labels-idx1-ubyte',
    root+'/t10k-images-idx3-ubyte', root+'/t10k-labels-idx1-ubyte')
trainx, trainy, testx, testy = spyker.to_torch(trainx, trainy, testx, testy)
train = DataLoader(TensorDataset(trainx, trainy), batch_size=batch)
test = DataLoader(TensorDataset(testx, testy), batch_size=batch)

Since the sparse interface runs on CPU, we can skip the device arguement in the `Transform` module and change `rankcode` to `rankcvt` to get sparse spikes. 

In [ ]:
class Transform:
    def __init__(self):
        self.filter = spyker.LoG(3, [.5, 1, 2], pad=3)
    
    def __call__(self, input):
        return sparse.code(spyker.threshold(self.filter(input), .01), 15)

We skip the device arguement in the `Network` module too. A few other changes need to be applied to the network so that it will be able to process sparse input.

In [ ]:
class Network:
    def __init__(self):
        self.conv1 = spyker.Conv(6, 50, 5, pad=2)
        self.conv2 = spyker.Conv(50, 100, 3, pad=1)
        self.conv1.stdpconfig = [spyker.STDPConfig(.004, -.003)]
        self.conv2.stdpconfig = [spyker.STDPConfig(.004, -.003)]
    
    def train1(self, input):
        output = sparse.inhibit(self.conv1(input, 16))
        self.conv1.stdp(input, sparse.convwta(output, 3, 5))
        
    def train2(self, input):
        input = sparse.pool(self.conv1(input, 16), 2)
        output = sparse.inhibit(self.conv2(input, 5))
        self.conv2.stdp(input, sparse.convwta(output, 1, 8))
    
    def __call__(self, input):
        input = sparse.pool(self.conv1(input, 16), 2)
        input = sparse.pool(self.conv2(input, 5), 3)
        return (15 - spyker.to_torch(sparse.gather(input))).flatten(1)

The rest of the code doesn't need to change.

In [ ]:
def Total(network, transform, dataset):
    data_total, target_total = [], []
    for data, target in dataset:
        data_total.append(network(transform(data)).cpu())
        target_total.append(target)
    return torch.cat(data_total), torch.cat(target_total)

In [ ]:
def Update(config):
    rate = config.negative / config.positive
    pos = min(config.positive * 2, .1)
    config.positive, config.negative = pos, pos * rate

In [ ]:
transform = Transform()
network = Network()

In [ ]:
from tqdm.notebook import tqdm

for i, (data, _) in enumerate(tqdm(train, "Training Layer 1")):
    if (i + 1) % 10 == 0: Update(network.conv1.stdpconfig[0])
    network.train1(transform(data))
spyker.quantize(network.conv1.kernel, 0, .5, 1)

for i, (data, _) in enumerate(tqdm(train, "Training Layer 2")):
    if (i + 1) % 10 == 0: Update(network.conv2.stdpconfig[0])
    network.train2(transform(data))
spyker.quantize(network.conv2.kernel, 0, .5, 1);

In [ ]:
train_data, train_target = Total(network, transform, train)
test_data, test_target = Total(network, transform, test)

In [ ]:
pca = PCA(n_components=200).fit(train_data, train_target)
train_data, test_data = pca.transform(train_data), pca.transform(test_data)

In [ ]:
target = SVC(C=2.4).fit(train_data, train_target).predict(test_data)
accuracy = (torch.tensor(target) == test_target).sum() / len(test_target)
print(f"Final Accuracy: {accuracy * 100 :.2f}%")

We can see that the accuracy is nearly the same as before and it runs fairly fast even though it runs on the CPU.